## Compute frequencies of **adjacent pairs** of symbols
### The vocabulary will be updated during the algorithm.

In [ ]:
import re, collections
from collections import defaultdict

def update_freq(vocab):
    pairs = defaultdict(int)
    for word, freq in vocab.items():
        #symbols = [w for w in word]
        symbols = word.split()
        #print(symbols)
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

## The first stage output of constucting our vocabulary.
### Define the training data. ( "-" represent the end of the sentence or row )

In [ ]:
train_data = {'l o w -': 5, 'l o w e r -': 2, 'n e w e s t -': 6, 'w i d e s t -': 3}
update_freq(train_data)

defaultdict(int,
            {('d', 'e'): 3,
             ('e', 'r'): 2,
             ('e', 's'): 9,
             ('e', 'w'): 6,
             ('i', 'd'): 3,
             ('l', 'o'): 7,
             ('n', 'e'): 6,
             ('o', 'w'): 7,
             ('r', '-'): 2,
             ('s', 't'): 9,
             ('t', '-'): 9,
             ('w', '-'): 5,
             ('w', 'e'): 8,
             ('w', 'i'): 3})

### Choosing the most repetetive pattern.

In [ ]:
def detect_best(MyDict):
  ref=0
  sel=('','')
  for tup,num in MyDict.items():
    if num > ref :
      sel=tup
      ref=num
  return sel

sel=detect_best(update_freq(train_data))
sel

('e', 's')

In [ ]:
def reconst_corpus(pair,data):
  for string,num in data.items():
    if pair[0]+' '+pair[1] in string:
      data[string.replace(pair[0]+' '+pair[1],pair[0]+pair[1])] = data.pop(string)
  return data

In [ ]:
reconst_corpus(('t', '-'),train_data)

{'l o w -': 5, 'l o w e r -': 2, 'n e w e s t-': 6, 'w i d e s t-': 3}

In [ ]:
train_data = {'l o w -': 5, 'l o w e r -': 2, 'n e w e s t -': 6, 'w i d e s t -': 3}
# Define two empty dictionaries.
bpe_stages = {}
bpe_subword = {}

num_merges = 10

for i in range(num_merges):
    print(f"\nIteration {i+1}:\n")
    pairs = update_freq(train_data) # appending to the vocabulary.
    best = detect_best(pairs)
    train_data = reconst_corpus(best, train_data) # update the corpus.
    
    bpe_stages[best] = i
    bpe_subword[best[0] + best[1]] = best
    
    print("new merge: {}".format(best))
    print("train data: {}".format(train_data))
    


Iteration 1:

new merge: ('e', 's')
train data: {'l o w -': 5, 'l o w e r -': 2, 'n e w es t -': 6, 'w i d es t -': 3}

Iteration 2:

new merge: ('es', 't')
train data: {'l o w -': 5, 'l o w e r -': 2, 'n e w est -': 6, 'w i d est -': 3}

Iteration 3:

new merge: ('est', '-')
train data: {'l o w -': 5, 'l o w e r -': 2, 'n e w est-': 6, 'w i d est-': 3}

Iteration 4:

new merge: ('l', 'o')
train data: {'n e w est-': 6, 'w i d est-': 3, 'lo w -': 5, 'lo w e r -': 2}

Iteration 5:

new merge: ('lo', 'w')
train data: {'n e w est-': 6, 'w i d est-': 3, 'lo w e r -': 2, 'low -': 5}

Iteration 6:

new merge: ('n', 'e')
train data: {'w i d est-': 3, 'lo w e r -': 2, 'low -': 5, 'ne w est-': 6}

Iteration 7:

new merge: ('ne', 'w')
train data: {'w i d est-': 3, 'lo w e r -': 2, 'low -': 5, 'new est-': 6}

Iteration 8:

new merge: ('new', 'est-')
train data: {'w i d est-': 3, 'lo w e r -': 2, 'low -': 5, 'newest-': 6}

Iteration 9:

new merge: ('low', '-')
train data: {'w i d est-': 3, 'lo w e

In [ ]:
bpe_stages

{('e', 's'): 0,
 ('es', 't'): 1,
 ('est', '-'): 2,
 ('l', 'o'): 3,
 ('lo', 'w'): 4,
 ('low', '-'): 8,
 ('n', 'e'): 5,
 ('ne', 'w'): 6,
 ('new', 'est-'): 7,
 ('w', 'i'): 9}

In [ ]:
bpe_subword

{'es': ('e', 's'),
 'est': ('es', 't'),
 'est-': ('est', '-'),
 'lo': ('l', 'o'),
 'low': ('lo', 'w'),
 'low-': ('low', '-'),
 'ne': ('n', 'e'),
 'new': ('ne', 'w'),
 'newest-': ('new', 'est-'),
 'wi': ('w', 'i')}

# Test the procedure for the new given word : slowest
## Return set of symbol pairs in a word. Word is represented as a tuple of symbols (symbols being variable-length strings).

In [ ]:
def get_pairs(word):
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char, char))
        prev_char = char
    return pairs

In [ ]:
w=word = tuple("lowest") + ('-',)
get_pairs(w)

{('e', 's'), ('l', 'o'), ('o', 'w'), ('s', 't'), ('t', '-'), ('w', 'e')}

## choosing by priority in the bpe_stage dictionary.

In [ ]:
def return_best(MyDict,pair):
  for i in range(len(MyDict)):
    if list(MyDict)[i] in pair:
      return list(MyDict)[i]

## merging the suggested pairs together 

In [ ]:
def merge_subwords(word,dual):
  word=list(word)
  L=len(word)
  for i in range(L-1):
    if word[i:i+2]==list(dual):
      word.pop(i+1)
      word[i]=dual[0]+dual[1]
  return tuple(word)

In [ ]:
merge_subwords(w,('e', 's'))

['l', 'o', 'w', 'es', 't', '-']

## Encode word based on list of BPE merge operations, which are applied consecutively

In [ ]:
def Forward(oov_word):

    word = tuple(oov_word) + ('-',) # convert the given oov vord to tuple form.
    print(f"word split into characters:{word}")
    # break the tuple into potential pairs.
    pairs_list = get_pairs(word)    
    #init iteration
    iteration = 0
    while True:
        iteration += 1
        print(f"\nIteration :{iteration}\n")
        print("Here are the potential pairs to be merged: {}".format(pairs_list))

        dual = return_best(bpe_stages,pairs_list)
        print("candidate for merging: {}".format(dual))
        #check out
        if dual not in bpe_stages:
            print("\nCandidate not in BPE merges, algorithm stops.")
            break
        #reassign the new word.
        word = merge_subwords(word,dual)
        print("word after merging: {}".format(word))

        #if all the pairs were found in the bpe_stages.
        if len(word) == 1:
            break
        else:
            pairs_list = get_pairs(word)

    # remove the end of the sentence (-)
    if word[-1] == '-':
        word = word[:-1]
    elif word[-1].endswith('-'):
        word = word[:-1] + (word[-1].replace('-',''),)
   
    return word

In [ ]:
a=Forward("lowest")

word split into characters:('l', 'o', 'w', 'e', 's', 't', '-')

Iteration :1

Here are the potential pairs to be merged: {('o', 'w'), ('s', 't'), ('t', '-'), ('l', 'o'), ('e', 's'), ('w', 'e')}
candidate for merging: ('e', 's')
word after merging: ('l', 'o', 'w', 'es', 't', '-')

Iteration :2

Here are the potential pairs to be merged: {('o', 'w'), ('t', '-'), ('es', 't'), ('w', 'es'), ('l', 'o')}
candidate for merging: ('es', 't')
word after merging: ('l', 'o', 'w', 'est', '-')

Iteration :3

Here are the potential pairs to be merged: {('o', 'w'), ('w', 'est'), ('est', '-'), ('l', 'o')}
candidate for merging: ('est', '-')
word after merging: ('l', 'o', 'w', 'est-')

Iteration :4

Here are the potential pairs to be merged: {('o', 'w'), ('w', 'est-'), ('l', 'o')}
candidate for merging: ('l', 'o')
word after merging: ('lo', 'w', 'est-')

Iteration :5

Here are the potential pairs to be merged: {('w', 'est-'), ('lo', 'w')}
candidate for merging: ('lo', 'w')
word after merging: ('low', 'es